# Connect to GDrive

In [ ]:
### Grab 1 year of data first ####

In [ ]:
!pip install import-ipynb japanese-candlestick marketprofile ta
from google.colab import drive, auth
import import_ipynb, sys, pandas, requests, datetime, bs4, json,pandas_datareader, market_profile, ta
from google.auth import default
from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
from pandas.tseries.offsets import BDay
import patterns     # candlesticks
from patterns import *
from market_profile import MarketProfile
from ta import add_all_ta_features    # technical indicators
from ta.utils import dropna
from bs4 import BeautifulSoup
from dateutil import parser
import yfinance as yf

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Small Caps/Database
path_to_module = '/content/drive/MyDrive/Small Caps/Database'
sys.path.append(path_to_module)
auth.authenticate_user()
creds, _ = default()

# Symbols List

In [ ]:
symbols = pandas_datareader.nasdaq_trader.get_nasdaq_symbols()
symbols['length'] = symbols['NASDAQ Symbol'].apply(lambda a: len(str(a)))
symbols['dot'] = symbols['NASDAQ Symbol'].apply(lambda a: 0 if str(a).find('.') == -1 else 1)
symbols = symbols[(symbols.length < 5) & (symbols['dot'] == 0)]
symbols = symbols['NASDAQ Symbol']
symbols = symbols[20:22]

# Data Tables 1-2 : Technical Data

In [ ]:
dailytable, intratable = pandas.DataFrame(), pandas.DataFrame()

for symbol in symbols:

  # connect to Yahoo Finance API
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  # daily data
  querystring = {"interval":'1d',"symbol": symbol,"range":'1y',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()

  # variables
  today = datetime.datetime.today()
  ipo = datetime.datetime.fromtimestamp(response['chart']['result'][0]['meta']['firstTradeDate'])
  ex = response['chart']['result'][0]['meta']['exchangeName']
  curr = response['chart']['result'][0]['meta']['currency']
  v = response['chart']['result'][0]['indicators']['quote'][0]['volume'][-1]
  h = response['chart']['result'][0]['indicators']['quote'][0]['high'][-1]
  c = response['chart']['result'][0]['indicators']['quote'][0]['close'][-1]
  dv = v * c
  o = response['chart']['result'][0]['indicators']['quote'][0]['open'][-1]
  l = response['chart']['result'][0]['indicators']['quote'][0]['low'][-1]
  a_c = response['chart']['result'][0]['indicators']['adjclose'][0]['adjclose'][-1]
  pc = response['chart']['result'][0]['indicators']['quote'][0]['close'][-2]
  h_52wk = max(response['chart']['result'][0]['indicators']['quote'][0]['high'])
  l_52wk = min(response['chart']['result'][0]['indicators']['quote'][0]['low'])
  on_gap = (o/pc) - 1
  close_vs_high = (c/h) - 1
  close_vs_low = (c/l) - 1
  candle_clr = 'G' if c >= o else 'R'
  daily_move = (c/pc) - 1
  daily_range = (h/l) - 1

  # convert to dataframe
  data = pandas.DataFrame(
  {'ticker': symbol,
   'ipo_date': ipo.date(),
   'exchange': ex,
   'currency' : curr,
   'datefield': today.date(),
   'volume': v,
   'dollar_volume': dv,
   'high': h,
   'open': o,
   'low': l,
   'close': c,
   'adj_close': a_c,
   'prev_close': pc,
   'high_52wk': h_52wk,
   'low_52wk': l_52wk,
   'on_gap': on_gap,
   'close_vs_high': close_vs_high,
   'close_vs_low': close_vs_low,
   'candle_colour' : candle_clr,
   'daily_move': daily_move,
   'daily_range': daily_range
  },index = [0])

  # intraday data
  querystring = {"interval":'1m',"symbol": symbol,"range":'1d',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()

  # variables
  dt1 = response['chart']['result'][0]['timestamp']
  dt = []
  for i in dt1:
    dt.append(datetime.datetime.fromtimestamp(i))

  vol = response['chart']['result'][0]['indicators']['quote'][0]['volume']
  high = response['chart']['result'][0]['indicators']['quote'][0]['high']
  open = response['chart']['result'][0]['indicators']['quote'][0]['open']
  low = response['chart']['result'][0]['indicators']['quote'][0]['low']
  close = response['chart']['result'][0]['indicators']['quote'][0]['close']
  cndl_clr = 'G' if close >= open else 'R'

  dollar_vol = []
  for i in range(0, len(vol)):
    try:
      dollar_vol.append(close[i] * vol[i])
    except:
      dollar_vol.append(-1)

  # convert to dataframe
  i_data = pandas.DataFrame(
    {'ticker': symbol,
     'datefield': dt,
     'volume': vol,
     'dollar_volume' : dollar_vol,
     'high': high,
     'open': open,
     'low': low,
     'close': close,
     'candle_colour' : cndl_clr
    })

  i_data['prev_close'] = i_data.close.shift(periods = 1)
  i_data['cndl_move'] = (i_data.close/i_data.prev_close) - 1
  i_data['cndl_range'] = (i_data.high/i_data.low) - 1
  i_data['shorts_HOD_volume'] = i_data.volume[i_data.high == i_data.high.max()]
  i_data['dip_long_volume'] = i_data.volume[i_data.low == i_data.low.min()]
  i_data['pv'] = ((i_data.high + i_data.low + i_data.close) / 3) * i_data.volume
  i_data['pv_sum'] = i_data.pv.cumsum()
  i_data['vol_cumsum'] = i_data.volume.cumsum()
  i_data['vwap_cum'] = i_data.pv_sum / i_data.vol_cumsum
  i_data = i_data.drop(columns = ['pv','pv_sum','vol_cumsum'])

  # Grab intraday only (time conversion to EST ?)
  i_data = i_data[i_data.datefield >= datetime.datetime(today.year,today.month,today.day,9,0,0)]

  # Append each ticker onto master table
  dailytable = dailytable.append(data)
  intratable = intratable.append(i_data)

dailytable = dailytable[['ticker','ipo_date','exchange','currency','datefield','volume','dollar_volume','high',
                         'open','low','close','adj_close','prev_close','high_52wk','low_52wk','on_gap','close_vs_high',
                         'close_vs_low','candle_colour','daily_move','daily_range']]

intratable = intratable[['ticker','datefield','volume','dollar_volume','high','open','low','close','candle_colour','prev_close',
                   'cndl_move','cndl_range','shorts_HOD_volume','dip_long_volume','vwap_cum']]

# Append master tables to production tables
key_statistics = pandas.read_excel("key_statistics.xlsx")
if max(key_statistics.datefield) != max(dailytable.datefield):
  key_statistics = key_statistics.append(dailytable)
  key_statistics.to_excel("key_statistics.xlsx")

intraday_data = pandas.read_excel("intraday_data.xlsx")
if max(intraday_data.datefield) != max(intratable.datefield):
  intraday_data = intraday_data.append(intratable)

    # Last 30 Days
  intraday_data = intraday_data[intraday_data.datefield >= today - BDay(20)]
  intraday_data.to_excel("intraday_data.xlsx")

# Data Table 3: Financial Data

In [ ]:
financial_stats = pandas.DataFrame()

for symbol in symbols:
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-statistics"
  querystring = {"symbol": symbol,"region":"US","lang":"en-US","straddle":"true"}
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  response = requests.get(url, headers=headers, params=querystring).json()

  if response != {}:
    table1 = pandas.DataFrame(response['defaultKeyStatistics']).iloc[0]
    table2 = pandas.DataFrame(response['price']).iloc[0]
    table3 = pandas.DataFrame(response['financialData']).iloc[0]
    table4 = pandas.DataFrame(response['calendarEvents']).iloc[0]
    table4 = table4[['dividendDate','earnings']]
    table4.earnings = table4.earnings[0]['fmt']
    table5 = pandas.DataFrame(response['summaryDetail']).iloc[0]
    table6 = pandas.concat([table1,table2,table3,table4,table5])
    table6 = pandas.DataFrame(table6).T
    financial_stats = financial_stats.append(table6)

financial_stats['datefield'] = datetime.date.today()

# convert from timestamp to datefield
columns_to_convert = ['lastFiscalYearEnd','nextFiscalYearEnd','mostRecentQuarter','sharesShortPreviousMonthDate','lastSplitDate','lastDividendDate',
                      'dateShortInterest','regularMarketTime','postMarketTime','preMarketTime','exDividendDate','dividendDate']

for col in columns_to_convert:
  try:
    financial_stats[col] = financial_stats[col].apply(lambda a: datetime.datetime.fromtimestamp(a))
  except:
    financial_stats[col] = datetime.datetime(1901,1,1,11,59,59)

fin_stats = financial_stats.T.drop_duplicates().T # remove duplicate columns

# Append master table to production table
financial_stats = pandas.read_excel("financial_stats.xlsx")
financial_stats = financial_stats.append(fin_stats)
financial_stats.to_excel("financial_stats.xlsx")

# Data Table 4-8: Summary, Fund, Institution, Insider Data

In [ ]:
for symbol in symbols:
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary"
  querystring = {"symbol": symbol ,"region":"US"}
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  response = requests.get(url, headers=headers, params=querystring).json()

  # fund ownership
  fund_table = []
  for i in range(0,len(response['fundOwnership']['ownershipList'])):
    r = response['fundOwnership']['ownershipList'][i]['reportDate']['raw']
    o = response['fundOwnership']['ownershipList'][i]['organization']
    h = response['fundOwnership']['ownershipList'][i]['pctHeld']['raw']
    p = response['fundOwnership']['ownershipList'][i]['position']['raw']
    v = response['fundOwnership']['ownershipList'][i]['value']['raw']
    c = response['fundOwnership']['ownershipList'][i]['pctChange']['raw']
    fund_table.append((r,o,h,p,v,c))

  fund_table_df = pandas.DataFrame(fund_table,columns = ['reportDate','organization','holding','position','value','change'])
  fund_table_df['reportDate'] = fund_table_df.reportDate.apply(lambda a: datetime.datetime.fromtimestamp(a))
  fund_table_df = fund_table_df.sort_values(by = 'reportDate', ascending = False)
  fund_table_df = fund_table_df[fund_table_df.reportDate == max(fund_table_df.reportDate)]
  fund_table_df['symbol'] = symbol
  fund_table_df['datefield'] = datetime.date.today()

  # insider transactions
  ins_table = []
  for i in range(0,len(response['insiderTransactions']['transactions'])):
    f = response['insiderTransactions']['transactions'][i]['filerName']
    t = response['insiderTransactions']['transactions'][i]['transactionText']
    o = response['insiderTransactions']['transactions'][i]['ownership']
    d = response['insiderTransactions']['transactions'][i]['startDate']['raw']
    v = response['insiderTransactions']['transactions'][i]['value']['raw']
    p = response['insiderTransactions']['transactions'][i]['filerRelation']
    s = response['insiderTransactions']['transactions'][i]['shares']['raw']
    ins_table.append((f,t,o,d,v,p,s))

  ins_table_df = pandas.DataFrame(ins_table,columns = ['insider_name','insider_trans','insider_own','insider_date','insider_value','insider_pos','insider_shrs'])
  ins_table_df['insider_date'] = ins_table_df.insider_date.apply(lambda a: datetime.datetime.fromtimestamp(a))
  ins_table_df = ins_table_df.sort_values(by = 'insider_date', ascending = False)
  ins_table_df = ins_table_df[ins_table_df.insider_date == max(ins_table_df.insider_date)]
  ins_table_df['symbol'] = symbol
  ins_table_df['datefield'] = datetime.date.today()

  # convert to dataframe
  sum_data = pandas.DataFrame(
    { 'symbol': symbol,
      'datefield': datetime.date.today(),
      'sector': response['summaryProfile']['sector'],
      'city': response['summaryProfile']['city'],
      'state': response['summaryProfile']['state'],
      'industry': response['summaryProfile']['industry']
    },index = [0])

# Insider Transactions
hold_table = []
for i in range(0,len(response['insiderHolders']['holders'])):
  n = response['insiderHolders']['holders'][i]['name']
  r = response['insiderHolders']['holders'][i]['relation']
  d = response['insiderHolders']['holders'][i]['transactionDescription']
  t = response['insiderHolders']['holders'][i]['latestTransDate']['raw']
  try:
    p = response['insiderHolders']['holders'][i]['positionDirect']['raw']
  except:
    p = -1
  try:
    dd = response['insiderHolders']['holders'][i]['positionDirectDate']['raw']
  except:
    dd = 0
  hold_table.append((n,r,d,t,p,dd))

  hold_table_df = pandas.DataFrame(hold_table,columns = ['holder_name','holder_pos','holder_trx','trx_date','holder_value','holder_date'])
  hold_table_df['trx_date'] = hold_table_df.trx_date.apply(lambda a: datetime.datetime.fromtimestamp(a))
  hold_table_df['holder_date'] = hold_table_df.holder_date.apply(lambda a: datetime.datetime.fromtimestamp(a))
  hold_table_df = hold_table_df[hold_table_df.trx_date == max(hold_table_df.holder_date)]
  hold_table_df['symbol'] = symbol
  hold_table_df['datefield'] = datetime.date.today()

# Instutition Ownership
inst_table = []
for i in range(0,len(response['institutionOwnership']['ownershipList'])):
  d = response['institutionOwnership']['ownershipList'][i]['reportDate']['raw']
  o = response['institutionOwnership']['ownershipList'][i]['organization']
  p = response['institutionOwnership']['ownershipList'][i]['pctHeld']['raw']
  h = response['institutionOwnership']['ownershipList'][i]['position']['raw']
  v = response['institutionOwnership']['ownershipList'][i]['value']['raw']
  c = response['institutionOwnership']['ownershipList'][i]['pctChange']['raw']
  inst_table.append((d,o,p,h,v,c))

inst_table_df = pandas.DataFrame(inst_table,columns = ['reportDate','institution','inst_pctHeld','inst_pos','inst_value','inst_pctChange'])
inst_table_df['reportDate'] = inst_table_df.reportDate.apply(lambda a: datetime.datetime.fromtimestamp(a))
inst_table_df = inst_table_df[inst_table_df.reportDate == max(inst_table_df.reportDate)]
inst_table_df['symbol'] = symbol
inst_table_df['datefield'] = datetime.date.today()

# Append master table to production table
fund_table = pandas.read_excel("fund_table.xlsx")
if max(fund_table.reportDate) != max(fund_table_df.reportDate):
  fund_table = fund_table.append(fund_table_df)
  fund_table = fund_table[['reportDate','organization','holding','position','value','change','symbol','datefield']]
  fund_table.to_excel("fund_table.xlsx")

insider_table = pandas.read_excel("insider_table.xlsx")
if max(insider_table.insider_date) != max(ins_table_df.insider_date):
  insider_table = insider_table.append(ins_table_df)
  insider_table =	insider_table[['insider_name','insider_trans','insider_own','insider_date','insider_value','insider_pos','insider_shrs','symbol','datefield']]
  insider_table.to_excel("insider_table.xlsx")

summary_table = pandas.read_excel("summary_table.xlsx")
summary_table = summary_table.append(sum_data)
summary_table = summary_table[['symbol','datefield','sector','city','state','industry']]
summary_table = summary_table.drop_duplicates()
summary_table.to_excel("summary_table.xlsx")

insider_trx = pandas.read_excel("insider_trx.xlsx")
if max(insider_trx.trx_date) != max(hold_table_df.trx_date):
  insider_trx = insider_trx.append(hold_table_df)
  insider_trx = insider_trx[['holder_name','holder_pos','holder_trx','trx_date','holder_value','holder_date','symbol','datefield']]
  insider_trx.to_excel("insider_trx.xlsx")

inst_table = pandas.read_excel("institution_table.xlsx")
if max(inst_table.reportDate) != max(inst_table_df.reportDate):
  inst_table = inst_table.append(inst_table_df)
  inst_table = inst_table[['reportDate','institution','inst_pctHeld','inst_pos','inst_value','inst_pctChange','symbol','datefield']]
  inst_table.to_excel("institution_table.xlsx")

# Data Table 9: Candlesticks

In [ ]:
'''
supernova catalysts (tim bohen and sykes notes)
1. shorts get over aggressive
2. a BS catalyst in a PR
3. discord chat rooms bashing the company
4. hot sector
5. abnormal volume
6. low float
7. former runner
8. low price
'''

In [ ]:
candlesticks = pandas.DataFrame()

for symbol in symbols:
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  # daily data
  querystring = {"interval":'1d',"symbol": symbol,"range":'1y',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()

  # variables
  data = pandas.DataFrame(response['chart']['result'][0]['indicators']['quote'][0])
  data.columns = data.columns.str.capitalize()
  data['Symbol'] = symbol
  data['datefield'] = datetime.date.today()

  # candlesticks
  cndls = [Doji,DragonflyDoji,Engulfing, Hammer,HangingMan,Harami,
          InvertedHammer,LongleggedDoji,ShootingStar,ThreeWhiteSoldiers,ThreeBlackCrows]
  for c in cndls:
    data = c(data).compute_pattern()

  data = data.iloc[-1]
  candlesticks = candlesticks.append(data)

# Append master table to production table
candlestick_table = pandas.read_excel("candlestick_table.xlsx")
if max(candlestick_table.datefield) != max(candlesticks.datefield):
  candlestick_table = candlestick_table.append(candlesticks)
  candlestick_table.to_excel("candlestick_table.xlsx")

# Data Table 10-12: Support/Resistance & Market Profile

In [ ]:
sup_res, prof, mp_df = pandas.DataFrame(), pandas.DataFrame(), pandas.DataFrame()

for symbol in symbols:
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  # daily data
  querystring = {"interval":'1d',"symbol": symbol,"range":'1y',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()
  data = pandas.DataFrame(response['chart']['result'][0]['indicators']['quote'][0])
  data.columns = data.columns.str.capitalize()

  # market profile
  mp = MarketProfile(data)
  mp_slice = mp[data.index.min():data.index.max()]
  support_resistance = pandas.DataFrame(mp_slice.high_value_nodes)
  support_resistance['symbol'] = symbol
  support_resistance['datefield'] = datetime.date.today()
  profile = pandas.DataFrame(mp_slice.profile)
  profile['symbol'] = symbol
  profile['datefield'] = datetime.date.today()

  try:
    opl, oph = mp_slice.open_range()[0] , mp_slice.open_range()[1]
  except:
    opl,oph = -1, -1

  try:
    ibl,ibh = mp_slice.initial_balance()[0], mp_slice.initial_balance()[1]
  except:
    ibl,ibh = -1, -1

  # convert to dataframe
  mp = pandas.DataFrame(
    { 'symbol': symbol,
      'datefield': datetime.date.today(),
      'balanced_target': mp_slice.balanced_target,
      'value_area_low': mp_slice.value_area[0],
      'value_area_high': mp_slice.value_area[1],
      'profile_range_low': mp_slice.profile_range[0],
      'profile_range_high': mp_slice.profile_range[1],
      'poc_price': mp_slice.poc_price,
      'open_range_low': opl,
      'open_range_high': oph,
      'initial_bal_low': ibl,
      'initial_bal_high': ibh
    },index = [0])

  sup_res = sup_res.append(support_resistance)
  prof = prof.append(profile)
  mp_df = mp_df.append(mp)

sup_res = sup_res.reset_index()
prof = prof.reset_index()
mp_df = mp_df.reset_index()

# Append master table to production table
sup_res_table = pandas.read_excel("support_resistance_table.xlsx")
if max(sup_res_table.datefield) != max(sup_res.datefield):
  sup_res_table = sup_res_table.append(sup_res)
  sup_res_table = sup_res_table[['Close','Volume','symbol','datefield']]
  sup_res_table.to_excel("support_resistance_table.xlsx")

m_p_sum = pandas.read_excel("market_profile_summary.xlsx")
if max(m_p_sum.datefield) != max(prof.datefield):
  m_p_sum = m_p_sum.append(prof)
  m_p_sum = m_p_sum[['Close','Volume','symbol','datefield']]
  m_p_sum.to_excel("market_profile_summary.xlsx")

m_p_det = pandas.read_excel("market_profile_detail.xlsx")
if max(m_p_det.datefield) != max(mp_df.datefield):
  m_p_det = m_p_det.append(mp_df)
  m_p_det = m_p_det[['symbol', 'datefield', 'balanced_target', 'value_area_low','value_area_high', 'profile_range_low', 'profile_range_high',
                      'poc_price', 'open_range_low', 'open_range_high', 'initial_bal_low','initial_bal_high']]
  m_p_det.to_excel("market_profile_detail.xlsx")

# Data Table 13: Technical Indicators

In [ ]:
tech_ind = pandas.DataFrame()

for symbol in symbols:
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  # daily data
  querystring = {"interval":'1d',"symbol": symbol,"range":'1y',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()
  data = pandas.DataFrame(response['chart']['result'][0]['indicators']['quote'][0])
  data.columns = data.columns.str.capitalize()

  # technicals
  data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume",fillna=True)
  data['Symbol'] = symbol
  data['Datefield'] = datetime.date.today()
  data['cndl_clr'] = list(map(lambda c, o: 1 if c - o >= 0 else -1, data.Close, data.Open))
  data['vol_acc_ind'] = sum(data.Volume * data.cndl_clr)
  data['vol_acc_3d'] = data.vol_acc_ind.rolling(3).sum()
  data['vol_acc_4d'] = data.vol_acc_ind.rolling(4).sum()
  data['vol_acc_5d'] = data.vol_acc_ind.rolling(5).sum()
  data['vol_acc_10d'] = data.vol_acc_ind.rolling(10).sum()

  data['sma_10d'] = data.Close.rolling(10).mean()
  data['sma_20d'] = data.Close.rolling(20).mean()
  data['sma_50d'] = data.Close.rolling(50).mean()
  data['sma_100d'] = data.Close.rolling(100).mean()
  data['sma_200d'] = data.Close.rolling(200).mean()

  data['ave_vol_3d'] = data.Volume.rolling(3).mean()
  data['ave_vol_5d'] = data.Volume.rolling(5).mean()
  data['ave_vol_10d'] = data.Volume.rolling(10).mean()
  data['ave_vol_30d'] = data.Volume.rolling(30).mean()
  data['ave_vol_60d'] = data.Volume.rolling(60).mean()
  data['ave_vol_90d'] = data.Volume.rolling(90).mean()

  data['high_1d'] = data.High.rolling(1).max()
  data['high_3d'] = data.High.rolling(3).max()
  data['high_5d'] = data.High.rolling(5).max()
  data['high_10d'] = data.High.rolling(10).max()
  data['high_15d'] = data.High.rolling(15).max()
  data['high_20d'] = data.High.rolling(20).max()
  data['high_30d'] = data.High.rolling(30).max()
  data['high_60d'] = data.High.rolling(60).max()
  data['high_90d'] = data.High.rolling(90).max()
  data['high_120d'] = data.High.rolling(120).max()

  data['low_1d'] = data.Low.rolling(1).min()
  data['low_3d'] = data.Low.rolling(3).min()
  data['low_5d'] = data.Low.rolling(5).min()
  data['low_10d'] = data.Low.rolling(10).min()
  data['low_15d'] = data.Low.rolling(15).min()
  data['low_20d'] = data.Low.rolling(20).min()
  data['low_30d'] = data.Low.rolling(30).min()
  data['low_60d'] = data.Low.rolling(60).min()
  data['low_90d'] = data.Low.rolling(90).min()
  data['low_120d'] = data.Low.rolling(120).min()

  data = data.iloc[-1]
  tech_ind = tech_ind.append(data)

# Technicals
technicals = pandas.read_excel("technical_data.xlsx")
if max(tech_ind.Datefield) != max(technicals.Datefield):
  technicals = technicals.append(tech_ind)
  technicals = technicals[['Close', 'Volume', 'Open', 'High', 'Low', 'volume_adi', 'volume_obv',
          'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt',
          'volume_vwap', 'volume_mfi', 'volume_nvi', 'volatility_bbm',
          'volatility_bbh', 'volatility_bbl', 'volatility_bbw', 'volatility_bbp',
          'volatility_bbhi', 'volatility_bbli', 'volatility_kcc',
          'volatility_kch', 'volatility_kcl', 'volatility_kcw', 'volatility_kcp',
          'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
          'volatility_dch', 'volatility_dcm', 'volatility_dcw', 'volatility_dcp',
          'volatility_atr', 'volatility_ui', 'trend_macd', 'trend_macd_signal',
          'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast',
          'trend_ema_slow', 'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
          'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index', 'trend_dpo',
          'trend_kst', 'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
          'trend_ichimoku_base', 'trend_ichimoku_a', 'trend_ichimoku_b',
          'trend_stc', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_cci',
          'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b', 'trend_aroon_up',
          'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
          'trend_psar_down', 'trend_psar_up_indicator',
          'trend_psar_down_indicator', 'momentum_rsi', 'momentum_stoch_rsi',
          'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d', 'momentum_tsi',
          'momentum_uo', 'momentum_stoch', 'momentum_stoch_signal', 'momentum_wr',
          'momentum_ao', 'momentum_roc', 'momentum_ppo', 'momentum_ppo_signal',
          'momentum_ppo_hist', 'momentum_pvo', 'momentum_pvo_signal',
          'momentum_pvo_hist', 'momentum_kama', 'others_dr', 'others_dlr',
          'others_cr','Symbol','Datefield','vol_acc_3d','vol_acc_4d',
          'vol_acc_5d','vol_acc_10d','sma_10d','sma_20d','sma_50d','sma_100d','sma_200d',
          'ave_vol_3d','ave_vol_5d','ave_vol_10d','ave_vol_30d','ave_vol_60d','ave_vol_90d',
          'high_1d','high_3d','high_5d','high_10d','high_15d','high_20d','high_30d','high_60d',
          'high_90d','high_120d','low_1d','low_3d','low_5d','low_10d','low_15d','low_20d','low_30d','low_60d',
          'low_90d','low_120d']]
  technicals.to_excel("technical_data.xlsx")

# Data Table 14: SEC Filings

In [ ]:
%cd /content/drive/MyDrive/Small Caps/Functions
path_to_module = '/content/drive/MyDrive/Small Caps/Functions'
sys.path.append(path_to_module)

import sec_filings, sec_filings_keywords
from sec_filings import get_sec_filing
from sec_filings_keywords import get_sec_filing_keyword

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
forms_ = ['424b3','424b4','424b5','s-3','f-3','3','4','sc+13g','sc+13d','s-1','f-1']
forms_2 = ['6-k','8-k']
keywords = ['reverse split','reverse stock split',                                                                            # reverse split
            '3.01 Notice',                                                                                                    # delist notice
            'Item 5.02','Board of Directors','Departure of Director','Appointment of New Director','Election of Directors',   # director change
            'annual meeting of stockholders','annual meeting of shareholders','elected to the Board',                         # director change
            'Annual Meeting the Company’s stockholders',                                                                      # director change
            'Item 4.01','Appointment of Auditor','independent registered public accounting firm',                             # new accountant
            'LLP as auditor','independent auditor']
filings = pandas.DataFrame()
pandas.set_option('display.max_colwidth', None)

# sec filings
for form in forms_:
  links = get_sec_filing(form,0)
  for link in range(0,len(links)):
    data = [links[link][0],links[link][1],links[link][2],form]
    df = pandas.DataFrame(data).T
    df.columns = ['cik','datefield','url','filing_type']
    df['time'] = df.datefield.apply(lambda a: a.time())
    df['datefield'] = df.datefield.apply(lambda a: a.date())
    df['keyword'] = 'NA'
    df = df[['cik','datefield','time','url','filing_type','keyword']]
    filings = filings.append(df)

# sec filings, keywords
for form in forms_2:
  links = get_sec_filing(form,0)
  for word in keywords:
    symbolss = get_sec_filing_keyword(links, word)
    symbolss = pandas.DataFrame(symbolss, columns = ['cik','datefield','keyword'])
    symbolss['filing_type'] = form
    symbolss['time'] = symbolss.datefield.apply(lambda a: a.time())
    symbolss['datefield'] = symbolss.datefield.apply(lambda a: a.date())
    symbolss['url'] = 'NA'
    symbolss = symbolss[['cik','datefield','time','url','filing_type','keyword']]
    filings = filings.append(symbolss)

filings = filings.drop_duplicates(keep = 'first')

# convert cik to ticker
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
url = 'https://www.sec.gov/files/company_tickers.json'
r = requests.get(url,headers=headers)

with open("sample.html", "w") as html_file:
    html_file.write(r.text)

with open("sample.html", "r") as html_file:
    html = html_file.read()

idx_lb = [i for i in range(len(html)) if html.startswith("{", i)]
idx_lb = idx_lb[1:]
idx_ub = [i for i in range(len(html)) if html.startswith("}", i)]
idx_ub = idx_ub[:-1]
ticker_table = []

for lb,ub in zip(idx_lb,idx_ub):
  line_of_data = html[lb:ub]
  line_of_data_sp = list(line_of_data.split(","))
  idx1 = line_of_data_sp[0].find(':') + 1
  idx2 = line_of_data_sp[1].find(':') + 1
  cik = line_of_data_sp[0][idx1:]
  ticker = line_of_data_sp[1][idx2:].replace('"','')
  ticker_table.append((cik,ticker))

ticker_table = pandas.DataFrame(ticker_table,columns = ['cik','symbol'])
filings = filings.merge(ticker_table, how = 'inner', left_on = 'cik', right_on = 'cik')

# Change directory back
%cd /content/drive/MyDrive/Small Caps/Database
path_to_module = '/content/drive/MyDrive/Small Caps/Database'
sys.path.append(path_to_module)

# Append new file to existing datatable
sec_filings = pandas.read_excel("sec_filings_table.xlsx")
if max(sec_filings.datefield) != max(filings.datefield):
  sec_filings = sec_filings.append(filings)
  sec_filings = sec_filings[['cik','datefield','time','url','filing_type','keyword','symbol']]
  sec_filings.to_excel("sec_filings_table.xlsx")

# incorporation state

# Data Table 15: News

In [ ]:
news_table_df = pandas.DataFrame()
pandas.set_option('display.max_colwidth', None)

for symbol in symbols:
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/news/v2/list"
  querystring = {"region":"US","snippetCount":"28","s":symbol}
  payload = "Pass in the value of uuids field returned right in this endpoint to load the next page, or leave empty to load first page"
  headers = {"content-type": "text/plain","X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  response = requests.post(url, data=payload, headers=headers, params=querystring).json()

  for i in range(0, len(response['data']['main']['stream'])):
    datefield = response['data']['main']['stream'][i]['content']['pubDate']
    datefield = parser.parse(datefield)
    date_ = datetime.date(datefield.year,datefield.month,datefield.day)
    datefield = datetime.datetime(datefield.year,datefield.month,datefield.day,datefield.hour,datefield.minute,datefield.second)

    if date_ == datetime.date.today():
      try:
        # convert to dataframe
        news_table = pandas.DataFrame(
          { 'symbol': symbol,
            'datefield': datetime.date.today(),
            'storytitle': response['data']['main']['stream'][i]['content']['title'],
            'pub_time': datefield.time(),
            'storyurl': response['data']['main']['stream'][i]['content']['clickThroughUrl']['url'],
            'newsource': response['data']['main']['stream'][i]['content']['provider']['displayName']
          },index = [0])
        news_table_df = news_table_df.append(news_table)
      except:
        news_table = pandas.DataFrame(
          { 'symbol': symbol,
            'datefield': datetime.date.today(),
            'storytitle': response['data']['main']['stream'][i]['content']['title'],
            'pub_time': datefield.time(),
            'storyurl': response['data']['main']['stream'][i]['content']['previewUrl'],
            'newsource': response['data']['main']['stream'][i]['content']['provider']['displayName']
          },index = [0])
        news_table_df = news_table_df.append(news_table)

# Append new file to existing datatable
news = pandas.read_excel("news_table.xlsx")
if max(sec_filings.datefield) != max(filings.datefield):
  news = news.append(news_table_df)
  news = news[['symbol','datefield','storytitle','pub_time','storyurl','newsource']]
  news.to_excel("news_table.xlsx")

# Data Table 16: Market Table

In [ ]:
dailytable = pandas.DataFrame()

for symbol in ['SPY','DIA','QQQ', 'XLE', 'XLF', 'XLU', 'XLI', 'GDX', 'XLK', 'XLV',
               'XLY', 'XLP', 'XLB', 'XOP', 'IYR', 'XHB', 'ITB', 'VNQ', 'GDXJ',
               'IYE', 'OIH', 'XME', 'XRT', 'SMH', 'IBB', 'KBE', 'KRE', 'XTL']:

  # connect to Yahoo Finance API
  headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "apidojo-yahoo-finance-v1.p.rapidapi.com"}
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-chart"

  # daily data
  querystring = {"interval":'1d',"symbol": symbol,"range":'1y',"region":"US"}
  response = requests.get(url, headers=headers, params=querystring).json()

  # variables
  v = response['chart']['result'][0]['indicators']['quote'][0]['volume'][-1]
  h = response['chart']['result'][0]['indicators']['quote'][0]['high'][-1]
  c = response['chart']['result'][0]['indicators']['quote'][0]['close'][-1]
  dv = v * c
  o = response['chart']['result'][0]['indicators']['quote'][0]['open'][-1]
  l = response['chart']['result'][0]['indicators']['quote'][0]['low'][-1]
  a_c = response['chart']['result'][0]['indicators']['adjclose'][0]['adjclose'][-1]
  pc = response['chart']['result'][0]['indicators']['quote'][0]['close'][-2]
  h_52wk = max(response['chart']['result'][0]['indicators']['quote'][0]['high'])
  l_52wk = min(response['chart']['result'][0]['indicators']['quote'][0]['low'])

  # convert to dataframe
  data = pandas.DataFrame(
  {'symbol': symbol,
   'datefield': datetime.date.today(),
   'volume': v,
   'dollar_volume': dv,
   'high': h,
   'open': o,
   'low': l,
   'close': c,
   'adj_close': a_c,
   'prev_close': pc,
   'high_52wk': h_52wk,
   'low_52wk': l_52wk
  },index = [0])

  dailytable = dailytable.append(data)

for symbol in ['BTC-USD','ETH-USD','USDT-USD','BNB-USD','XRP-USD','SOL-USD','USDC-USD','ADA-USD','DOGE-USD','AVAX-USD']:

  btc = yf.download(symbol)
  pc = btc.Close.iloc[-2]
  h_52wk = btc.High.iloc[-365].max()
  l_52wk = btc.Low.iloc[-365].min()
  btc = btc.iloc[-1]

  # convert to dataframe
  data = pandas.DataFrame(
  {'symbol': symbol,
   'datefield': datetime.date.today(),
   'volume': btc.Volume,
   'dollar_volume': btc.Volume * btc.Close,
   'high': btc.High,
   'open': btc.Open,
   'low': btc.Low,
   'close': btc.Close,
   'adj_close': btc['Adj Close'],
   'prev_close': pc,
   'high_52wk': h_52wk,
   'low_52wk': l_52wk
  },index = [0])

  dailytable = dailytable.append(data)

# Append master tables to production tables
market_table = pandas.read_excel("market_table.xlsx")
if max(key_statistics.datefield) != max(dailytable.datefield):
  market_table = market_table.append(dailytable)
  market_table.to_excel("market_table.xlsx")

# Data Table 17: Chat Room Table

In [ ]:
url = "https://twitter241.p.rapidapi.com/search"
querystring = {"query":"$SHOT","count":"1000","type":"Top"}
headers = {"X-RapidAPI-Key": "1a4e1671fdmsh8b109f9960abd33p10d082jsnf2ee61df8b81","X-RapidAPI-Host": "twitter241.p.rapidapi.com"}
response = requests.get(url, headers=headers, params=querystring).json()
response['result']['timeline']['instructions'] # API is complex ...

#date
#ticker
#Chat_room_ind (reddit, twitter, stocktwits…)
#msg_vol
#likes_vol
#dislikes_vol
#comments_vol
#Sentiment
#User
#user_followers

# custom code to develop using:
# 1. twitter's developer API
# 2. reddit's developer API and scrape wallstreetbets
# 3. discord developer API

# Data Table 18: Short Squeeze Table

In [ ]:
# ticker
# date
#time-to-cover: shares short > 5x average daily volume
#shares short as a % of float > 10%
# no. shares short rising daily
#Low_float_ind
# Finding locates for shorts (determines share availability)
# Ensuring shorts locates are cheap
# short availability : https://www.quantconnect.com/datasets/us-equities-short-availability

# Data Table 19: Pattern Stats Table

In [ ]:
#ticker
#date
#cup_and_handle
#ascending_triangle
#descending_triangle
#converging_triangle
#stair_stepper
#fgd_breakout
#frd_breakdown
#multi_day_runner
#supernova
#cliff_dive
#double_top_ind
#double_bottom_ind
#morning_panic_ind
#morning_spike_ind
#dip_buy
#head_shoulders
#inverse_cup_handle
#reverse_stair_stepper
#dead_pump_bounce
#fgd_ind
#sgd_ind
#frd_ind
#srd_ind
#breakout_ind
#Breakdown_ind
#over_extended_ind
#lower_highs_ind
#higher_lows_ind
#consolidation_ind

# Resources

In [ ]:
https://www.quantconnect.com/datasets/nasdaq-data-link
https://www.quantconnect.com/datasets